In [4]:
import os
from pathlib import Path
from dotenv import load_dotenv
import findspark
env_path = Path(os.getenv("HOME")) / "data_engineering_zoomcamp_2025" / "5_batch" / ".env"
load_dotenv(dotenv_path=env_path)
findspark.init()

In [41]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [8]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [13]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [14]:
from pyspark.sql import types

In [15]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True)]
)

In [16]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [17]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [18]:
df = df.repartition(24)

In [19]:
df.write.parquet('fhvhv/2021/01')

AnalysisException: path file:/home/marc/data_engineering_zoomcamp_2025/5_batch/fhvhv/2021/01 already exists.

In [20]:
df = spark.read.parquet('fhvhv/2021/01')

In [22]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [31]:
from pyspark.sql import functions as F

In [36]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [37]:
crazy_stuff('B02884')

's/b44'

In [38]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [40]:
df \
.withColumn('pickup_date', F.to_date(df.pickup_datetime))\
.withColumn('dropoff_date', F.to_date(df.dropoff_datetime))\
.withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num))\
.select('hvfhs_license_num', 'base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')\
.show()

+-----------------+-------+-----------+------------+------------+------------+
|hvfhs_license_num|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------------+-------+-----------+------------+------------+------------+
|           HV0005|  e/9ce| 2021-01-01|  2021-01-01|          21|          55|
|           HV0005|  e/9ce| 2021-01-02|  2021-01-02|          42|         159|
|           HV0003|  e/b3b| 2021-01-01|  2021-01-01|         213|          32|
|           HV0003|  s/b13| 2021-01-01|  2021-01-01|         220|         235|
|           HV0003|  e/b38| 2021-01-02|  2021-01-02|         190|          61|
|           HV0003|  e/b32| 2021-01-02|  2021-01-02|          71|          61|
|           HV0003|  e/b35| 2021-01-02|  2021-01-02|         198|         198|
|           HV0003|  e/b38| 2021-01-01|  2021-01-01|         107|          48|
|           HV0003|  e/a39| 2021-01-02|  2021-01-02|          61|         177|
|           HV0005|  e/9ce| 2021-01-01|  2021-01-01|

In [29]:
df.select('hvfhs_license_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-----------------+-------------------+-------------------+------------+------------+
|hvfhs_license_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-----------------+-------------------+-------------------+------------+------------+
|           HV0003|2021-01-01 03:35:52|2021-01-01 03:46:17|         213|          32|
|           HV0003|2021-01-01 21:29:06|2021-01-01 21:49:01|         220|         235|
|           HV0003|2021-01-02 17:26:59|2021-01-02 17:40:43|         190|          61|
|           HV0003|2021-01-02 17:54:07|2021-01-02 18:07:12|          71|          61|
|           HV0003|2021-01-02 18:37:59|2021-01-02 18:46:57|         198|         198|
|           HV0003|2021-01-01 12:58:30|2021-01-01 13:16:56|         107|          48|
|           HV0003|2021-01-02 03:46:04|2021-01-02 03:53:44|          61|         177|
|           HV0003|2021-01-01 03:58:31|2021-01-01 04:03:15|          56|         196|
|           HV0003|2021-01-02 15:33:12|2021-01-02 15:5